In [1]:
from pathlib import Path
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
Path.cwd()

PosixPath('/Users/arthur/Documents/Astronomy/2019/Retrieval/Code/APOLLO')

In [3]:
from custom_types import Pathlike
from user_directories import USER_DIRECTORIES  # noqa: E402

RESULTS_DIRECTORY: Pathlike = USER_DIRECTORIES["results"]


results_directory: Path = Path.cwd() / RESULTS_DIRECTORY

In [4]:
test_results_directory: Path = (
    results_directory / "2M2236" / "HK+JWST_2M2236_logg-free_cloudy"
)

In [5]:
from xarray import load_dataset, Dataset

In [6]:
contribution_dataset: Dataset = load_dataset(
    test_results_directory / "2M2236.Piette.HK+G395H.cloudy.contributions.nc"
)

In [7]:
contribution_dataset

<xarray.Dataset> Size: 3MB
Dimensions:       (log_pressure: 70, wavelength: 469)
Coordinates:
  * wavelength    (wavelength) float64 4kB 1.481 1.484 1.487 ... 5.248 5.258
  * log_pressure  (log_pressure) float64 560B -3.954 -3.861 ... 2.361 2.454
Data variables:
    h2only        (log_pressure, wavelength) float64 263kB 8.484 8.583 ... 0.0
    h2o           (log_pressure, wavelength) float64 263kB 37.44 243.2 ... 0.0
    co            (log_pressure, wavelength) float64 263kB 2.491e-05 ... 0.0
    co2           (log_pressure, wavelength) float64 263kB 0.09984 ... 0.0
    ch4           (log_pressure, wavelength) float64 263kB 1.717 1.165 ... 0.0
    Lupu_alk      (log_pressure, wavelength) float64 263kB 3.742e-05 ... 0.0
    h2s           (log_pressure, wavelength) float64 263kB 7.415e-06 ... 0.0
    nh3           (log_pressure, wavelength) float64 263kB 184.2 462.2 ... 0.0
    total         (log_pressure, wavelength) float64 263kB 231.9 715.1 ... 0.0
    cloud         (log_pressure, wavelength) float64 263kB 0.0 0.0 ... 0.0006689
    gas           (log_pressure, wavelength) float64 263kB 231.9 715.1 ... 0.0
Attributes:
    title:    contribution_functions

In [8]:
contribution_dataset.get(["h2only", "h2o", "co"]).max()

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    h2only   float64 8B 1.784e+08
    h2o      float64 8B 1.753e+10
    co       float64 8B 1.337e+10

In [9]:
from typing import Final


COMPOSITE_COMPONENTS: Final[list[str]] = ["gas", "cloud", "total"]

In [10]:
np.nanmax(
    contribution_dataset.get(
        [
            data_variable
            for data_variable in contribution_dataset
            if data_variable not in COMPOSITE_COMPONENTS
        ]
    ).to_array()
)

17529807544.34574

In [11]:
data_dataset: Dataset = load_dataset(
    test_results_directory / "2M2236b_HK+G395H_R500.nc"
)

In [12]:
data_dataset

<xarray.Dataset> Size: 32kB
Dimensions:                (wavelength: 469)
Coordinates:
  * wavelength             (wavelength) float32 2kB 1.481 1.484 ... 5.248 5.258
    band                   (wavelength) <U10 19kB 'H' 'H' ... 'G395H_NRS2'
Data variables:
    wavelength_bin_starts  (wavelength) float32 2kB 1.479 1.482 ... 5.242 5.253
    wavelength_bin_ends    (wavelength) float32 2kB 1.482 1.485 ... 5.253 5.263
    flux                   (wavelength) float32 2kB 1.596e-10 ... 9.264e-11
    lower_errors           (wavelength) float32 2kB 5.318e-12 ... 2.24e-12
    upper_errors           (wavelength) float32 2kB 5.318e-12 ... 2.24e-12
    flux_with_noise        (wavelength) float32 2kB 1.596e-10 ... 9.264e-11
Attributes:
    title:    2M2236 b

In [18]:
from xarray import Variable


contribution_dataset = contribution_dataset.assign_coords(
    band=Variable(
        **{
            "dims": ["wavelength"],
            "data": data_dataset.band.to_numpy(),
            "attrs": {"bands": tuple(set(data_dataset.band.to_numpy()))},
        }
    )
)

In [20]:
contribution_dataset

<xarray.Dataset> Size: 3MB
Dimensions:       (log_pressure: 70, wavelength: 469)
Coordinates:
  * wavelength    (wavelength) float64 4kB 1.481 1.484 1.487 ... 5.248 5.258
  * log_pressure  (log_pressure) float64 560B -3.954 -3.861 ... 2.361 2.454
    band          (wavelength) <U10 19kB 'H' 'H' ... 'G395H_NRS2' 'G395H_NRS2'
Data variables:
    h2only        (log_pressure, wavelength) float64 263kB 8.484 8.583 ... 0.0
    h2o           (log_pressure, wavelength) float64 263kB 37.44 243.2 ... 0.0
    co            (log_pressure, wavelength) float64 263kB 2.491e-05 ... 0.0
    co2           (log_pressure, wavelength) float64 263kB 0.09984 ... 0.0
    ch4           (log_pressure, wavelength) float64 263kB 1.717 1.165 ... 0.0
    Lupu_alk      (log_pressure, wavelength) float64 263kB 3.742e-05 ... 0.0
    h2s           (log_pressure, wavelength) float64 263kB 7.415e-06 ... 0.0
    nh3           (log_pressure, wavelength) float64 263kB 184.2 462.2 ... 0.0
    total         (log_pressure, wavelength) float64 263kB 231.9 715.1 ... 0.0
    cloud         (log_pressure, wavelength) float64 263kB 0.0 0.0 ... 0.0006689
    gas           (log_pressure, wavelength) float64 263kB 231.9 715.1 ... 0.0
Attributes:
    title:    contribution_functions

In [25]:
from apollo.spectrum.Spectrum_measured_using_xarray import DataSpectrum


DataSpectrum(contribution_dataset).get_bands("G395H_NRS1")

<xarray.Dataset> Size: 1MB
Dimensions:       (log_pressure: 70, wavelength: 165)
Coordinates:
  * wavelength    (wavelength) float64 1kB 2.876 2.882 2.888 ... 3.994 4.002
  * log_pressure  (log_pressure) float64 560B -3.954 -3.861 ... 2.361 2.454
    band          (wavelength) <U10 7kB 'G395H_NRS1' ... 'G395H_NRS1'
Data variables:
    h2only        (log_pressure, wavelength) float64 92kB 17.37 17.3 ... 0.0 0.0
    h2o           (log_pressure, wavelength) float64 92kB 6.011e+03 ... 0.0
    co            (log_pressure, wavelength) float64 92kB 5.099e-05 ... 0.0
    co2           (log_pressure, wavelength) float64 92kB 37.87 75.95 ... 0.0
    ch4           (log_pressure, wavelength) float64 92kB 17.2 83.31 ... 0.0 0.0
    Lupu_alk      (log_pressure, wavelength) float64 92kB 7.6e-05 ... 0.0
    h2s           (log_pressure, wavelength) float64 92kB 0.03056 ... 0.0
    nh3           (log_pressure, wavelength) float64 92kB 7.115e+04 ... 0.0
    total         (log_pressure, wavelength) float64 92kB 7.724e+04 ... 0.0
    cloud         (log_pressure, wavelength) float64 92kB 0.0 0.0 ... 0.002835
    gas           (log_pressure, wavelength) float64 92kB 7.724e+04 ... 0.0
Attributes:
    title:    contribution_functions

In [26]:
contribution_dataset = contribution_dataset.rename_dims({"log_pressure": "pressure"})

In [31]:
contribution_dataset.to_netcdf("test_contribution_output.nc")